#### Weather Pattern Correlation 
- Precipitation: PRISM vs ECMWF and SMYLE
- Q700: ERA-I vs ECMWF and SMYLE
- MSL : ERA-I vs ECMWF and SMYLE

note: all variables in interpolated to coarse SMYLE grid
Ming Ge Nov. 2024

In [1]:
import sys
import datetime
import scipy
from scipy.stats import pearsonr
from scipy.spatial import Delaunay
#import scipy.spatial.qhull as qhull
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import glob 
import os.path
import cartopy 
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# inorder to import my pathon module, add
# $PYTHONPATH for jupyter for module
sys.path.append("/glade/u/home/mingge/PYTHON")
import coexist_func 
from importlib import reload
reload(coexist_func)

from coexist_func import cal_season_cli_prism
from coexist_func import ax_coastlines_country
from coexist_func import cal_season_ano_prism
from coexist_func import plot_prec_12

In [2]:
# Interpolation functions
def interp_weights(xy, uv,d=2):
    tri = Delaunay(xy)
    simplex = tri.find_simplex(uv)
    vertices = np.take(tri.simplices, simplex, axis=0)
    temp = np.take(tri.transform, simplex, axis=0)
    delta = uv - temp[:, d]
    bary = np.einsum('njk,nk->nj', temp[:, :d, :], delta)
    return vertices, np.hstack((bary, 1 - bary.sum(axis=1, keepdims=True)))

def interpolate(values, vtx, wts):
    return np.einsum('nj,nj->n', np.take(values, vtx), wts)

def pattern_corr_2d(arr1, arr2):
    """
    Calculate 2D pattern correlation between arr1 and arr2, 
    handling NaN values.
    """

    # Remove NaN values from both arrays
    mask = ~np.isnan(arr1) & ~np.isnan(arr2)
    arr1_masked = arr1[mask]
    arr2_masked = arr2[mask]

    # Calculate correlation coefficient
    return np.corrcoef(arr1_masked, arr2_masked)[0, 1]


In [3]:
# centroid_ana.ibynb, wt_hist_smyle_era5.ipynb
sCentroids='/glade/u/home/mingge/WTing_existing-centroids/CONUS-WTs_existing_centroids.npz'
DATA=np.load(sCentroids)
rgrCentroids=DATA['rgrCentroids']               # (12, 3, 24, 74)
rgiWTs=range(1, rgrCentroids.shape[0]+1, 1)     # number of WT 

lat_s = DATA['rgrLatWT'].min() - 6
lat_e = DATA['rgrLatWT'].max() + 2
lon_s = DATA['rgrLonWT'].min()  
lon_e = DATA['rgrLonWT'].max() + 6
#print('lat:', lat_s , '-', lat_e, 'lon:', lon_s, '-', lon_e)

n_wt = 12

col_s = ['WT' + str(x) for x in np.arange(1, n_wt+1).tolist()]

# create dataframe containing pattern correlation data among 
# PRISM, ERAI, ECMWF and SMYLE datasets
df_corr_prec = pd.DataFrame(np.zeros((2, n_wt)), 
                            columns = col_s,
                           index = ['ECMWF', 'SMYLE'])

# Create a 2D DataFrame using copy() for a deep copy
df_corr_q700 = df_corr_prec.copy()
df_corr_msl = df_corr_prec.copy()
 

#### PREC

In [4]:
# SMTLE prec
flnm_cli = '/glade/campaign/mmm/c3we/mingge/COEXIST/SMYLE/prec_clim_large.nc'
with  xr.open_dataset(flnm_cli) as ds_cli:
    prec_cli = ds_cli.PRECT

n_lat = prec_cli.shape[0]
n_lon = prec_cli.shape[1]

n_mem = 20
for mem in range(1, n_mem+1, 1):
    mem_s = str(mem).zfill(2)
    flnm_wt = '/glade/campaign/mmm/c3we/mingge/COEXIST/SMYLE/prec_wt_' + mem_s + '.nc'
    
    with  xr.open_dataset(flnm_wt) as ds_prec:
        prec_wt = ds_prec.__xarray_dataarray_variable__
        if mem == 1:
            prec_avg = prec_wt.copy()
        else:
            prec_avg = prec_avg + prec_wt

prec_smyle = (prec_avg/float(n_mem) - prec_cli)/prec_cli*100

lon_grid_smyle, lat_grid_smyle = np.meshgrid(prec_smyle.lon, prec_smyle.lat)
lon_grid_smyle_flatten = lon_grid_smyle.flatten()
lat_grid_smyle_flatten = lat_grid_smyle.flatten()

n_lat = lon_grid_smyle.shape[0]
n_lon = lon_grid_smyle.shape[1]

In [5]:
# PRISM PREC
dir_prism = '/glade/campaign/mmm/c3we/mingge/COEXIST/PRISM/'
flnm_mean = dir_prism + 'prec_clim_1979-2014.nc'
flnm_prec = dir_prism + 'prec_wt.nc'

with  xr.open_dataset(flnm_mean) as ds:
    prec_mean = ds.PR

with  xr.open_dataset(flnm_prec) as ds:
    prec_xr = ds.__xarray_dataarray_variable__

prec_prism = (prec_xr - prec_mean)/prec_mean*100
# prec_prism(wt: 12, rlat: 605, rlon: 1405)

lon_grid_prism, lat_grid_prism = np.meshgrid(prec_prism.rlon.data+360, prec_prism.rlat.data)

# create gregridding weights
# Remap SMYLE to Centroid
points = np.array([lon_grid_prism.flatten(), lat_grid_prism.flatten()]).transpose()
vtx, wts = interp_weights(points, np.append(lon_grid_smyle_flatten[:,None], lat_grid_smyle_flatten[:,None], axis=1))

prec_prism_new = np.zeros((n_wt, n_lat, n_lon))
for nw in range(n_wt):
    prec_prism_new[nw] = interpolate(prec_prism[nw].data.flatten(), vtx, wts).reshape(n_lat, n_lon) 
    # pattern correlation calculation
    df_corr_prec.loc['SMYLE'][col_s[nw]] = pattern_corr_2d(prec_prism_new[nw], prec_smyle[nw].data)
    

In [6]:
# ECMWF prec
dir_ecmwf = '/glade/campaign/mmm/c3we/mingge/COEXIST/ECMWF/'
n_mem = 25
for mem in range(1, n_mem+1,1):
    mem_s = str(mem).zfill(2)
    flnm_wt = dir_ecmwf + 'prec_ano_mem' + mem_s + '.nc'
    
    with  xr.open_dataset(flnm_wt) as ds_prec:
        prec_wt = ds_prec.__xarray_dataarray_variable__*100
        if mem == 1:
            prec_avg = prec_wt.copy()
        else:
            prec_avg = prec_avg + prec_wt
prec_ecmwf = prec_avg/n_mem
lon_grid_ecmwf, lat_grid_ecmwf = np.meshgrid(prec_ecmwf.lon.data+360, prec_ecmwf.lat.data)

# create gregridding weights
# Remap SMYLE to Centroid
points = np.array([lon_grid_ecmwf.flatten(), lat_grid_ecmwf.flatten()]).transpose()
vtx, wts = interp_weights(points, np.append(lon_grid_smyle_flatten[:,None], lat_grid_smyle_flatten[:,None], axis=1)) 

In [7]:
prec_ecmwf_new = np.zeros((n_wt, n_lat, n_lon))
for nw in range(n_wt):
    prec_ecmwf_new[nw] = interpolate(prec_ecmwf[nw].data.flatten(), vtx, wts).reshape(n_lat, n_lon)  
     # pattern correlation calculation
    df_corr_prec.loc['ECMWF'][col_s[nw]] = pattern_corr_2d(prec_prism_new[nw], prec_ecmwf_new[nw])

#c = plt.pcolormesh(prec_smyle.lon, prec_smyle.lat, prec_ecmwf_new[1], cmap='RdBu_r', vmin = -100, vmax = 100)
#plt.colorbar(c)
#plt.show()
#prec_ecmwf[1].plot(cmap='RdBu_r', vmin = -100, vmax = 100)
 

#### Q700

In [8]:
# Q700 SMYLE
flnm_cli = '/glade/campaign/mmm/c3we/mingge/COEXIST/SMYLE/q700_clim_large.nc'
with  xr.open_dataset(flnm_cli) as ds_cli:
    prec_cli = ds_cli['Q700']

n_lat = prec_cli.shape[0]
n_lon = prec_cli.shape[1]

for mem in range(1, 20+1,1):
    mem_s = str(mem).zfill(2)
    flnm_wt = '/glade/campaign/mmm/c3we/mingge/COEXIST/SMYLE/q700_wt_' + mem_s + '.nc'
    #print(flnm_wt)
    
    with  xr.open_dataset(flnm_wt) as ds_prec:
        prec_wt = ds_prec.__xarray_dataarray_variable__
        if mem == 1:
            prec_avg = prec_wt.copy()
        else:
            prec_avg = prec_avg + prec_wt

q700_smyle = (prec_avg*0.05 - prec_cli)/prec_cli*100

lon_grid_smyle, lat_grid_smyle = np.meshgrid(q700_smyle.lon, q700_smyle.lat)
lon_grid_smyle_flatten = lon_grid_smyle.flatten()
lat_grid_smyle_flatten = lat_grid_smyle.flatten()
n_lat = lon_grid_smyle.shape[0]
n_lon = lon_grid_smyle.shape[1]

In [9]:
#### 2 Q700 ERAI
flnm_q700_erai = '/glade/campaign/mmm/c3we/mingge/COEXIST/ERAI/Q700/q700_ano.nc'
with xr.open_dataset(flnm_q700_erai) as ds:
    q700_erai = ds.__xarray_dataarray_variable__
    
lon_grid_erai, lat_grid_erai = np.meshgrid(q700_erai.lon.data, q700_erai.lat)

# create gregridding weights
# Remap SMYLE to Centroid
points = np.array([lon_grid_erai.flatten(), lat_grid_erai.flatten()]).transpose()
vtx, wts = interp_weights(points, np.append(lon_grid_smyle_flatten[:,None], lat_grid_smyle_flatten[:,None], axis=1))

In [10]:
q700_erai_new = np.zeros((n_wt, n_lat, n_lon))
for nw in range(n_wt):
    q700_erai_new[nw] = interpolate(q700_erai[nw].data.flatten(), vtx, wts).reshape(n_lat, n_lon)  

#c = plt.pcolormesh(q700_smyle.lon, q700_smyle.lat, q700_erai_new[1], cmap='RdBu_r', vmin = -100, vmax = 100)
#plt.colorbar(c)
#plt.show()
#q700_erai[1].plot(cmap='RdBu_r', vmin = -100, vmax = 100)

In [11]:
# Q700 ECMWF 
n_mem = 25
dir_wt = '/glade/campaign/mmm/c3we/mingge/COEXIST/ECMWF/'
flnm_q700_ecmwf_cli = '/glade/campaign/mmm/c3we/mingge/COEXIST/ECMWF/q700_clim_215day_eachMem.nc'
with  xr.open_dataset(flnm_q700_ecmwf_cli) as ds:
    var = ds.__xarray_dataarray_variable__[:n_mem]
    q700_cli = var.sel(g0_lat_3 = slice(lat_e, lat_s), g0_lon_4=slice(lon_s, lon_e))
q700_mean = q700_cli.mean(dim='member')

for mem in range(1, n_mem+1,1):
    mem_s = str(mem).zfill(2)
    flnm_wt = dir_wt + 'q700_wt_' + mem_s + '.nc'
    #print(flnm_wt)
    
    with  xr.open_dataset(flnm_wt) as ds:
        q700_wt = ds.__xarray_dataarray_variable__

        if mem == 1:
            q700_avg = q700_wt.copy()
        else:
            q700_avg = q700_avg + q700_wt

q700_ecmwf = (q700_avg/25.0 - q700_mean)/q700_mean*100
lon_grid_ecmwf, lat_grid_ecmwf = np.meshgrid(q700_cli.g0_lon_4.data+360, q700_cli.g0_lat_3.data)

# create gregridding weights
# Remap SMYLE to Centroid
points = np.array([lon_grid_ecmwf.flatten(), lat_grid_ecmwf.flatten()]).transpose()
vtx, wts = interp_weights(points, np.append(lon_grid_smyle_flatten[:,None], lat_grid_smyle_flatten[:,None], axis=1)) 

In [12]:
q700_ecmwf_new = np.zeros((n_wt, n_lat, n_lon))
for nw in range(n_wt):
    q700_ecmwf_new[nw] = interpolate(q700_ecmwf[nw].data.flatten(), vtx, wts).reshape(n_lat, n_lon)  
    df_corr_q700.loc['ECMWF'][col_s[nw]] = pattern_corr_2d(q700_erai_new[nw], q700_ecmwf_new[nw])
    df_corr_q700.loc['SMYLE'][col_s[nw]] = pattern_corr_2d(q700_erai_new[nw], q700_smyle[nw].data)

#print(df_corr_q700)

#c = plt.pcolormesh(q700_smyle.lon, q700_smyle.lat, q700_ecmwf_new[1], cmap='RdBu_r', vmin = -100, vmax = 100)
#plt.colorbar(c)
#plt.show()
#q700_ecmwf[1].plot(cmap='RdBu_r', vmin = -100, vmax = 100)

#### MSL

In [13]:
# SLP SMYLE
flnm_cli = '/glade/campaign/mmm/c3we/mingge/COEXIST/SMYLE/psl_clim_large.nc'
with  xr.open_dataset(flnm_cli) as ds_cli:
    prec_cli = ds_cli.PSL

n_lat = prec_cli.shape[0]
n_lon = prec_cli.shape[1]

for mem in range(1, 21,1):
    mem_s = str(mem).zfill(2)
    flnm_wt = '/glade/campaign/mmm/c3we/mingge/COEXIST/SMYLE/psl_wt_' + mem_s + '.nc'
    
    with  xr.open_dataset(flnm_wt) as ds_prec:
        prec_wt = ds_prec.__xarray_dataarray_variable__
        if mem == 1:
            prec_avg = prec_wt.copy()
        else:
            prec_avg = prec_avg + prec_wt

msl_smyle = (prec_avg*0.05 - prec_cli)/prec_cli*100

lon_grid_smyle, lat_grid_smyle = np.meshgrid(msl_smyle.lon, msl_smyle.lat)
lon_grid_smyle_flatten = lon_grid_smyle.flatten()
lat_grid_smyle_flatten = lat_grid_smyle.flatten()
n_lat = lon_grid_smyle.shape[0]
n_lon = lon_grid_smyle.shape[1]

In [14]:
#### MSL ERAI
flnm_q700_erai = '/glade/campaign/mmm/c3we/mingge/COEXIST/ERAI/MSL/msl_ano.nc'
with xr.open_dataset(flnm_q700_erai) as ds:
    msl_erai = ds.__xarray_dataarray_variable__

lon_grid_erai, lat_grid_erai = np.meshgrid(msl_erai.lon.data, msl_erai.lat)

# create gregridding weights
# Remap SMYLE to Centroid
points = np.array([lon_grid_erai.flatten(), lat_grid_erai.flatten()]).transpose()
vtx, wts = interp_weights(points, np.append(lon_grid_smyle_flatten[:,None], lat_grid_smyle_flatten[:,None], axis=1))

In [15]:
msl_erai_new = np.zeros((n_wt, n_lat, n_lon))
for nw in range(n_wt):
    msl_erai_new[nw] = interpolate(msl_erai[nw].data.flatten(), vtx, wts).reshape(n_lat, n_lon)  

#c = plt.pcolormesh(msl_smyle.lon, msl_smyle.lat, msl_erai_new[1], cmap='RdBu_r', vmin = -1, vmax = 1)
#plt.colorbar(c)
#plt.show()
#msl_erai[1].plot(cmap='RdBu_r', vmin = -1, vmax = 1)

In [16]:
# msl ECMWF
n_mem = 25
dir_wt = '/glade/campaign/mmm/c3we/mingge/COEXIST/ECMWF/'
flnm_q700_ecmwf_cli = '/glade/campaign/mmm/c3we/mingge/COEXIST/ECMWF/mslp_clim_215day_eachMem.nc'
with  xr.open_dataset(flnm_q700_ecmwf_cli) as ds:
    var = ds.__xarray_dataarray_variable__[:n_mem]
    msl_cli = var.sel(g0_lat_2 = slice(lat_e, lat_s), g0_lon_3=slice(lon_s, lon_e))
msl_mean = msl_cli.mean(dim='member')

for mem in range(1, n_mem+1,1):
    mem_s = str(mem).zfill(2)
    flnm_wt = dir_wt + 'mslp_wt_' + mem_s + '.nc'
    
    with  xr.open_dataset(flnm_wt) as ds:
        msl_wt = ds.__xarray_dataarray_variable__

        if mem == 1:
            msl_avg = msl_wt.copy()
        else:
            msl_avg = msl_avg + msl_wt

msl_ecmwf = (msl_avg/25.0 - msl_mean)/msl_mean*100
lon_grid_ecmwf, lat_grid_ecmwf = np.meshgrid(msl_mean.g0_lon_3.data+360, msl_mean.g0_lat_2.data)

# create gregridding weights
# Remap SMYLE to Centroid
points = np.array([lon_grid_ecmwf.flatten(), lat_grid_ecmwf.flatten()]).transpose()
vtx, wts = interp_weights(points, np.append(lon_grid_smyle_flatten[:,None], lat_grid_smyle_flatten[:,None], axis=1)) 

In [17]:
msl_ecmwf_new = np.zeros((n_wt, n_lat, n_lon))
for nw in range(n_wt):
    msl_ecmwf_new[nw] = interpolate(msl_ecmwf[nw].data.flatten(), vtx, wts).reshape(n_lat, n_lon)  
    df_corr_msl.loc['ECMWF'][col_s[nw]] = pattern_corr_2d(msl_erai_new[nw], msl_ecmwf_new[nw])
    df_corr_msl.loc['SMYLE'][col_s[nw]] = pattern_corr_2d(msl_erai_new[nw], msl_smyle[nw].data)

pd.options.display.float_format = "{:,.3f}".format
print('1 PRECIPITATION of PRISM against ECMWF and SMYLE')
print(df_corr_prec)

print('')
print('')
print('2 Q700 of ERAI against ECMWF and SMYLE')
print(df_corr_q700)

print('')
print('')
print('3 MSL of ERAI against ECMWF and SMYLE')
print(df_corr_msl)

#c = plt.pcolormesh(msl_smyle.lon, msl_smyle.lat, msl_ecmwf_new[1], cmap='RdBu_r', vmin = -1, vmax = 1)
#plt.colorbar(c)
#plt.show()
#msl_ecmwf[1].plot(cmap='RdBu_r', vmin = -1, vmax = 1)

1 PRECIPITATION of PRISM against ECMWF and SMYLE
        WT1   WT2   WT3   WT4   WT5   WT6   WT7   WT8   WT9   WT10  WT11  WT12
ECMWF 0.388 0.559 0.763 0.795 0.505 0.394 0.382 0.802 0.326  0.068 0.539 0.308
SMYLE 0.430 0.550 0.649 0.816 0.558 0.382 0.294 0.808 0.289 -0.124 0.529 0.445


2 Q700 of ERAI against ECMWF and SMYLE
        WT1   WT2   WT3   WT4   WT5   WT6   WT7   WT8   WT9  WT10  WT11  WT12
ECMWF 0.898 0.856 0.877 0.839 0.845 0.400 0.834 0.667 0.575 0.739 0.866 0.740
SMYLE 0.952 0.909 0.824 0.899 0.832 0.784 0.947 0.741 0.900 0.897 0.888 0.722


3 MSL of ERAI against ECMWF and SMYLE
        WT1   WT2   WT3   WT4   WT5   WT6   WT7   WT8   WT9  WT10  WT11  WT12
ECMWF 0.977 0.915 0.394 0.804 0.811 0.250 0.655 0.926 0.909 0.859 0.982 0.584
SMYLE 0.979 0.924 0.779 0.949 0.940 0.793 0.964 0.979 0.922 0.984 0.978 0.766
